In [12]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import os

def divide_into_chunks(input_file, chunk_size):
    with open(input_file, 'rb') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            yield chunk

def encrypt_chunk(key, chunk):
    cipher = AES.new(key, AES.MODE_GCM)
    ciphertext, tag = cipher.encrypt_and_digest(chunk)
    return cipher.nonce + tag + ciphertext

def decrypt_chunk(key, encrypted_chunk):
    nonce = encrypted_chunk[:16]
    tag = encrypted_chunk[16:32]
    ciphertext = encrypted_chunk[32:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    decrypted_chunk = cipher.decrypt_and_verify(ciphertext, tag)
    return decrypted_chunk

def encrypt_aes_key(aes_key, rsa_public_key):
    cipher_rsa = PKCS1_OAEP.new(rsa_public_key)
    encrypted_key = cipher_rsa.encrypt(aes_key)
    return encrypted_key


def decrypt_aes_key(encrypted_aes_key, rsa_private_key):
    cipher_rsa = PKCS1_OAEP.new(rsa_private_key)
    aes_key = cipher_rsa.decrypt(encrypted_aes_key)
    return aes_key


def generate_aes_key():
    return get_random_bytes(32)


def generate_rsa_key_pair():
    return RSA.generate(2048)

if __name__ == "__main__":
    input_file = 'large_file.txt'
    chunk_size = 1024 * 1024

    rsa_key_pair = generate_rsa_key_pair()
    rsa_public_key = rsa_key_pair.publickey()
    rsa_private_key = rsa_key_pair

    chunk_generator = divide_into_chunks(input_file, chunk_size)

    encrypted_chunks = []
    for chunk in chunk_generator:
        aes_key = generate_aes_key()
        encrypted_aes_key = encrypt_aes_key(aes_key, rsa_public_key)
        encrypted_chunk = encrypt_chunk(aes_key, chunk)
        encrypted_chunks.append((encrypted_aes_key, encrypted_chunk))

    output_file = 'reassembled_file.txt'
    with open(output_file, 'wb') as f:
        for encrypted_aes_key, encrypted_chunk in encrypted_chunks:
            aes_key = decrypt_aes_key(encrypted_aes_key, rsa_private_key)
            decrypted_chunk = decrypt_chunk(aes_key, encrypted_chunk)
            f.write(decrypted_chunk)

    print(f"File reassembled: {output_file}")


SyntaxError: invalid decimal literal (number.py, line 139)